Please run those two cells before running the Notebook!

As those plotting settings are standard throughout the book, we do not show them in the book every time we plot something.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
# FIX: Use the official public API path from pandas.errors
from pandas.errors import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# feel free to modify, for example, change the context to "notebook"
sns.set_theme(context="talk", style="whitegrid", 
              palette="colorblind", color_codes=True, 
              rc={"figure.figsize": [12, 8]})

# Chapter 4 - Exploring Financial Time Series Data

## 4.1 Outlier detection using rolling statistics

### How to do it...

1. Import the libraries:

In [ ]:
import pandas as pd 
import yfinance as yf

2. Download Tesla's stock prices from 2019-2020 and calculate simple returns:

In [ ]:
df = yf.download("TSLA", 
                 start="2019-01-01", 
                 end="2020-12-31",
                 progress=False)

# FIX: Use the 'Close' column, which now contains the adjusted close price
df["rtn"] = df["Close"].pct_change()
df = df[["rtn"]].copy()
df.head()


3. Calculate the rolling mean and standard deviation:

In [ ]:
df_rolling = df[["rtn"]].rolling(window=21) \
                        .agg(["mean", "std"])
df_rolling.columns = df_rolling.columns.droplevel()

4. Join the rolling data back to the initial DataFrame:

In [ ]:
df = df.join(df_rolling)
df.head()

5. Calculate the upper and lower thresholds:

In [ ]:
import yfinance as yf

# Step 1: Download the data
df = yf.download("TSLA", 
                 start="2019-01-01", 
                 end="2020-12-31",
                 progress=False,
                 auto_adjust=True)

# Step 2: FIX - Calculate the returns and create the 'rtn' column first
df["rtn"] = df["Close"].pct_change()

# Step 3: FIX - Now calculate the rolling mean and std from the 'rtn' column
df['mean'] = df['rtn'].rolling(window=30).mean()
df['std'] = df['rtn'].rolling(window=30).std()

# Step 4: The original code for the bands will now work
N_SIGMAS = 3
df["upper"] = df["mean"] + N_SIGMAS * df["std"]
df["lower"] = df["mean"] - N_SIGMAS * df["std"]

# Display the first few rows of the result
df.head()

6. Identify the outliers using the previously calculated thresholds:

In [ ]:
df["outlier"] = (
    (df["rtn"] > df["upper"]) | (df["rtn"] < df["lower"])
)

7. Plot the returns together with the thresholds and mark the outliers:

In [ ]:
fig, ax = plt.subplots()

df[["rtn", "upper", "lower"]].plot(ax=ax)
ax.scatter(df.loc[df["outlier"]].index, 
           df.loc[df["outlier"], "rtn"], 
           color="black", label="outlier")
ax.set_title("Tesla's stock returns")
ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_4_1", dpi=200)

### There's more

Define a function for identify outliers using the steps described in the previous section.

In [ ]:
def identify_outliers(df, column, window_size, n_sigmas):
    """Function for identifying outliers using rolling statistics"""
    
    # Make a copy to avoid modifying the original DataFrame
    df_copy = df[[column]].copy()
    
    # FIX: Calculate rolling stats and assign them directly as new columns
    df_copy["mean"] = df_copy[column].rolling(window=window_size).mean()
    df_copy["std"] = df_copy[column].rolling(window=window_size).std()
    
    # FIX: Calculate upper and lower bands on this new DataFrame
    df_copy["upper"] = df_copy["mean"] + n_sigmas * df_copy["std"]
    df_copy["lower"] = df_copy["mean"] - n_sigmas * df_copy["std"]
    
    # Return the boolean series indicating outliers
    return (df_copy[column] > df_copy["upper"]) | (df_copy[column] < df_copy["lower"])

# The function call remains the same
identify_outliers(df, "rtn", 21, 3)

## 4.2 Outlier detection with the Hampel filter

### How to do it...

1. Import the libraries:

In [ ]:
# FIX: Install the sktime library
!pip install sktime

In [ ]:
import yfinance as yf
from sktime.transformations.series.outlier_detection import HampelFilter

2. Download Tesla's stock prices from 2019-2020 and calculate simple returns:

In [ ]:
df = yf.download("TSLA", 
                 start="2019-01-01", 
                 end="2020-12-31",
                 progress=False)
df["rtn"] = df["Adj Close"].pct_change()

3. Instantiate the `HampelFilter` class and use it for detecting the outliers:

In [ ]:
hampel_detector = HampelFilter(window_length=10, 
                               return_bool=True)
df["outlier"] = hampel_detector.fit_transform(df["Adj Close"])
df.head()

4. Plot Tesla's stock price and mark the outliers:

In [ ]:
fig, ax = plt.subplots()

df[["Adj Close"]].plot(ax=ax)
ax.scatter(df.loc[df["outlier"]].index, 
           df.loc[df["outlier"], "Adj Close"], 
           color="black", label="outlier")
ax.set_title("Tesla's stock price")
ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_4_2", dpi=200)

### There's more

1. Identify the outliers among the stock returns:

In [ ]:
df["outlier_rtn"] = hampel_detector.fit_transform(df["rtn"])
df.head()

2. Plot Tesla's daily returns and mark the outliers:

In [ ]:
fig, ax = plt.subplots()

df[["rtn"]].plot(ax=ax)
ax.scatter(df.loc[df["outlier_rtn"]].index, 
           df.loc[df["outlier_rtn"], "rtn"], 
           color="black", label="outlier")
ax.set_title("Tesla's stock returns")
ax.legend(loc="center left", bbox_to_anchor=(1, 0.5))

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_4_3", dpi=200)

3. Investigate the overlap in outliers identified for the prices and returns:

In [ ]:
df.query("outlier == True and outlier_rtn == True").round(2)

## 4.3 Detecting changepoints in time series

### How to do it...

1. Import the libraries:

In [ ]:
import yfinance as yf
from kats.detectors.cusum_detection import CUSUMDetector
from kats.consts import TimeSeriesData

2. Download Apple's stock price from 2020:

In [ ]:
df = yf.download("AAPL", 
                 start="2020-01-01", 
                 end="2020-12-31", 
                 progress=False)
df["Adj Close"].plot(title="Apple's stock in 2020");

3. Keep only the adjusted close price, reset the index and rename the columns:

In [ ]:
df = df[["Adj Close"]].reset_index(drop=False)
df.columns = ["time", "price"]

4. Convert the DataFrame into a `TimeSeriesData` object:

In [ ]:
tsd = TimeSeriesData(df)

5. Instantiate and run the changepoint detector:

In [ ]:
cusum_detector = CUSUMDetector(tsd)
change_points = cusum_detector.detector(
    change_directions=["increase"]
)
cusum_detector.plot(change_points)

6. Investigate the detected changepoint in more detail:

In [ ]:
point, meta = change_points[0]
point

In [ ]:
meta.__dict__

### There's more

1. Narrow down the window in which we want to search for the changepoint:

In [ ]:
change_points = cusum_detector.detector(change_directions=["increase"], 
                                        interest_window=[200, 250])
cusum_detector.plot(change_points)


2. Use another algorithm to detect changepoints (`RobustStatDetector`): 

In [ ]:
from kats.detectors.robust_stat_detection import RobustStatDetector

robust_detector = RobustStatDetector(tsd)
change_points = robust_detector.detector()
robust_detector.plot(change_points)

3. Change the `RobustStatDetector`'s default arguments:

In [ ]:
robust_detector = RobustStatDetector(tsd)
changepoints = robust_detector.detector(p_value_cutoff = 0.01, comparison_window=-5)
robust_detector.plot(changepoints)

## 4.4 Detecting trends in time series

### How to do it...

1. Import the libraries:

In [ ]:
import yfinance as yf
from kats.consts import TimeSeriesData
from kats.detectors.trend_mk import MKDetector

2. Download NVIDIA's stock prices from 2020:

In [ ]:
df = yf.download("NVDA", 
                 start="2020-01-01", 
                 end="2020-12-31", 
                 progress=False)
df["Adj Close"].plot(title="NVIDIA's stock in 2020");

3. Keep only the adjusted close price, reset the index and rename the columns:

In [ ]:
df = df[["Adj Close"]].reset_index(drop=False)
df.columns = ["time", "price"]

4. Convert the DataFrame into a `TimeSeriesData` object:

In [ ]:
tsd = TimeSeriesData(df)

5. Instantiate and run the trend detector:

In [ ]:
trend_detector = MKDetector(tsd, threshold=0.9)
time_points = trend_detector.detector(
    direction="up", 
    window_size=30
)

6. Plot the detected time points:

In [ ]:
trend_detector.plot(time_points)

7. Investigate the detected points in more detail:

In [ ]:
len(time_points)

Inspect a single time point:

In [ ]:
cp, meta = time_points[0]

In [ ]:
cp

In [ ]:
meta.__dict__

## 4.5 Detecting patterns in a time series using the Hurst exponent

### How to do it...

1. Import the libraries:

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd

2. Download S&P 500's historical prices from the years 2000-2019:

In [ ]:
df = yf.download("^GSPC", 
                 start="2000-01-01", 
                 end="2019-12-31", 
                 progress=False)
df["Adj Close"].plot(title="S&P 500 (years 2000-2019)")

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_4_9", dpi=200);

3. Define a function calculating the Hurst exponent:

In [ ]:
def get_hurst_exponent(ts, max_lag=20):
    """Returns the Hurst Exponent of the time series"""
    
    lags = range(2, max_lag)

    # standard deviations of the lagged differences
    tau = [np.std(np.subtract(ts[lag:], ts[:-lag])) for lag in lags]

    # calculate the slope of the log plot -> the Hurst Exponent
    hurst_exp = np.polyfit(np.log(lags), np.log(tau), 1)[0]

    return hurst_exp

4. Calculate the values of the Hurst exponent using different values for the `max_lag` parameter:

In [ ]:
for lag in [20, 100, 250, 500, 1000]:
    hurst_exp = get_hurst_exponent(df["Adj Close"].values, lag)
    print(f"Hurst exponent with {lag} lags: {hurst_exp:.4f}")

5. Narrow down the data to the years 2005-2007 and calculate the exponents one more time:

In [ ]:
df.loc["2005":"2007", "Adj Close"].plot(title="S&P 500 (years 2005-2007)")

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_4_10", dpi=200);

In [ ]:
shorter_series = df.loc["2005":"2007", "Adj Close"].values
for lag in [20, 100, 250, 500]:
    hurst_exp = get_hurst_exponent(shorter_series, lag)
    print(f"Hurst exponent with {lag} lags: {hurst_exp:.4f}")

## 4.6 Investigating stylized facts of asset returns

### How to do it...

1. Import the libraries:

In [ ]:
import pandas as pd 
import numpy as np
import yfinance as yf
import seaborn as sns 
import scipy.stats as scs
import statsmodels.api as sm
import statsmodels.tsa.api as smt

2. Download the S&P 500 data and calculate the returns:

In [ ]:
df = yf.download("^GSPC", 
                 start="2000-01-01", 
                 end="2020-12-31",
                 progress=False)

df = df[["Adj Close"]].rename(
    columns={"Adj Close": "adj_close"}
)
df["log_rtn"] = np.log(df["adj_close"]/df["adj_close"].shift(1))
df = df[["adj_close", "log_rtn"]].dropna()
df

#### Fact 1 - Non-Gaussian distribution of returns

1. Calculate the Normal PDF using the mean and standard deviation of the observed returns:

In [ ]:
r_range = np.linspace(min(df["log_rtn"]), 
                      max(df["log_rtn"]), 
                      num=1000)
mu = df["log_rtn"].mean()
sigma = df["log_rtn"].std()
norm_pdf = scs.norm.pdf(r_range, loc=mu, scale=sigma)   

2. Plot the histogram and the Q-Q Plot:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 8))

# histogram
sns.distplot(df.log_rtn, kde=False, 
             norm_hist=True, ax=ax[0])                                    
ax[0].set_title("Distribution of S&P 500 returns", 
                fontsize=16)                                                    
ax[0].plot(r_range, norm_pdf, "g", lw=2, 
           label=f"N({mu:.2f}, {sigma**2:.4f})")
ax[0].legend(loc="upper left");

# Q-Q plot
qq = sm.qqplot(df.log_rtn.values, line="s", ax=ax[1])
ax[1].set_title("Q-Q plot", fontsize = 16)

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_4_11", dpi=200);

3. Print the summary statistics of the log returns:

In [ ]:
jb_test = scs.jarque_bera(df["log_rtn"].values)

print("---------- Descriptive Statistics ----------")
print("Range of dates:", min(df.index.date), "-", max(df.index.date))
print("Number of observations:", df.shape[0])
print(f"Mean: {df.log_rtn.mean():.4f}")
print(f"Median: {df.log_rtn.median():.4f}")
print(f"Min: {df.log_rtn.min():.4f}")
print(f"Max: {df.log_rtn.max():.4f}")
print(f"Standard Deviation: {df.log_rtn.std():.4f}")
print(f"Skewness: {df.log_rtn.skew():.4f}")
print(f"Kurtosis: {df.log_rtn.kurtosis():.4f}") 
print(f"Jarque-Bera statistic: {jb_test[0]:.2f} with p-value: {jb_test[1]:.2f}")

#### Fact 2 - Volatility Clustering

1. Run the following code to visualize the log returns series:

In [ ]:
(
    df["log_rtn"]
    .plot(title="Daily S&P 500 returns", figsize=(10, 6))
)

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_4_12", dpi=200);

#### Fact 3 - Absence of autocorrelation in returns

1. Define the parameters for creating the autocorrelation plots:

In [ ]:
N_LAGS = 50
SIGNIFICANCE_LEVEL = 0.05

2. Run the following code to create ACF plot of log returns:

In [ ]:
acf = smt.graphics.plot_acf(df["log_rtn"], 
                            lags=N_LAGS, 
                            alpha=SIGNIFICANCE_LEVEL)

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_4_13", dpi=200);

#### Fact 4 - Small and decreasing autocorrelation in squared/absolute returns

1. Create the ACF plots:

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 10))

smt.graphics.plot_acf(df["log_rtn"]**2, lags=N_LAGS, 
                      alpha=SIGNIFICANCE_LEVEL, ax=ax[0])
ax[0].set(title="Autocorrelation Plots",
          ylabel="Squared Returns")

smt.graphics.plot_acf(np.abs(df["log_rtn"]), lags=N_LAGS, 
                      alpha=SIGNIFICANCE_LEVEL, ax=ax[1])
ax[1].set(ylabel="Absolute Returns",
          xlabel="Lag")

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_4_14", dpi=200);

#### Fact 5 - Leverage effect

1. Calculate volatility measures as moving standard deviations

In [ ]:
df["rolling_std_252"] = df[["log_rtn"]].rolling(window=252).std()
df["rolling_std_21"] = df[["log_rtn"]].rolling(window=21).std()

2. Plot all the series:

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(18, 15), 
                       sharex=True)

df["adj_close"].plot(ax=ax[0])
ax[0].set(title="S&P 500 time series",
          ylabel="Price ($)")

df["log_rtn"].plot(ax=ax[1])
ax[1].set(ylabel="Log returns")

df["rolling_std_252"].plot(ax=ax[2], color="r", 
                           label="Rolling Volatility 252d")
df["rolling_std_21"].plot(ax=ax[2], color="g", 
                           label="Rolling Volatility 21d")
ax[2].set(ylabel="Moving Volatility",
          xlabel="Date")
ax[2].legend()

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_4_15", dpi=200);

### There's more

1. Download and preprocess the prices of the S&P 500 and VIX:

In [ ]:
df = yf.download(["^GSPC", "^VIX"], 
                 start="2000-01-01", 
                 end="2020-12-31",
                 progress=False)
df = df[["Adj Close"]]
df.columns = df.columns.droplevel(0)
df = df.rename(columns={"^GSPC": "sp500", "^VIX": "vix"})

2. Calculate the log returns (we can just as well use percentage change-simple returns):

In [ ]:
df["log_rtn"] = np.log(df["sp500"] / df["sp500"].shift(1))
df["vol_rtn"] = np.log(df["vix"] / df["vix"].shift(1))
df.dropna(how="any", axis=0, inplace=True)

3. Plot a scatterplot with the returns on the axes and fit a regression line to identify the trend:

In [ ]:
corr_coeff = df.log_rtn.corr(df.vol_rtn)

ax = sns.regplot(x="log_rtn", y="vol_rtn", data=df, 
                 line_kws={"color": "red"})
ax.set(title=f"S&P 500 vs. VIX ($\\rho$ = {corr_coeff:.2f})",
       ylabel="VIX log returns",
       xlabel="S&P 500 log returns")

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_4_16", dpi=200);